# Data Science Jobs Analyze


Group:
* Anchal
* Angela
* Larissa
* Fabio
* Felipe

This program extracts data from Indeed.ca website to analyze main caracteristics 
of Data Science Job opportunities in Toronto

In [247]:
#################### IMPORT LIBRARIES ####################

In [248]:
#import bs4
import numpy as np
import pandas as pd
import re 
import requests
import datetime
from bs4 import BeautifulSoup

In [249]:
#################### SEARCH ARGUMENTS ####################

In [250]:
input_job = "Data Scientist"
input_quote = False # add quotation marks("") to your input_job
input_city = "" # leave empty if input_city is not specified
input_state = "Canada"
sign = "+"
BASE_URL_indeed =  'https://ca.indeed.com/' #'http://www.indeed.com'


In [251]:
##### Function for Transform searching keywords #####
def transform(input,sign, quote = False):
    syntax = input.replace(" ", sign)
    if quote == True:
        syntax = ''.join(['"', syntax, '"'])
    return(syntax)

In [252]:
################# GENERATING BASE INDEED URL #################

In [253]:

if not input_city: # if (input_city is "")
    url_indeed_list = [ BASE_URL_indeed, '/jobs?q=', transform(input_job, sign, input_quote),
                    '&l=', input_state]
    url_indeed = ''.join(url_indeed_list)
else: # input_city is not ""
    url_indeed_list = [ BASE_URL_indeed, '/jobs?q=', transform(input_job, sign, input_quote),
                    '&l=', transform(input_city, sign), '%2C+', input_state]
    url_indeed = ''.join(url_indeed_list)

print(url_indeed)


https://ca.indeed.com//jobs?q=Data+Scientist&l=Canada


In [254]:
# get the HTML code from the URL
rawcode_indeed = requests.get(url_indeed)

# Choose "lxml" as parser
soup_indeed = BeautifulSoup(rawcode_indeed.text, "lxml")


In [255]:
# total number of results
num_total_indeed = soup_indeed.find(
                        id = 'searchCount').contents[0].split()[-2]

print("total results:")
print(soup_indeed.find(id = 'searchCount').contents[0])
print(soup_indeed.find(id = 'searchCount').contents[0].split()[-2])
num_total_indeed = re.sub("[^0-9]","", num_total_indeed) # remove non-numeric characters in the string
num_total_indeed = int(num_total_indeed)
#print(num_total_indeed)

# total number of pages
num_pages_indeed = int(np.ceil(num_total_indeed/10.0))
#print(num_pages_indeed)

# create an empty dataframe
df_base = pd.DataFrame()
# the date for today
now = datetime.datetime.now()
now_str = now.strftime("%m/%d/%Y")
now_str_name=now.strftime('%m%d%Y')

total results:

        Page 1 of 1,042 jobs
1,042


In [256]:
############ function to clean text before scraping ########### 

In [257]:
def text_cleaner(dirty_text):

    text = dirty_text

    lines = (line.strip() for line in text.splitlines()) # break into lines

    chunks = (phrase.strip() for line in lines for phrase in line.split("  ")) # break multi-headlines into a line each
    
    text = ''.join(chunk for chunk in chunks if chunk).encode('utf-8') # Get rid of all blank lines and ends of line
    
    # Now clean out all of the unicode junk (this line works great!!!)
    
    try:
        text = text.decode('unicode_escape').encode('ascii', 'ignore') # Need this as some websites aren't formatted
    except:                                                            # in a way that this works, can occasionally throw
        return                                                         # an exception
   
    #print(text)
    text = text.decode('utf-8')
    text = re.sub("[^a-zA-Z+3]"," ", text)  # Now get rid of any terms that aren't words (include 3 for d3.js)
                                             # Also include + for C++
    text = re.sub(r"([a-z])([A-Z])", r"\1 \2", text) # Fix spacing issue from merged words
    
    text = text.lower().split()  # Go to lower case and split them apart
    
    
   # stop_words = set(stopwords.words("english")) # Filter out any stop words
   # text = [w for w in text if not w in stop_words]
    
    text = list(set(text)) # Last, just get the set of these. Ignore counts (we are just looking at whether a term existed
                           # or not on the website)
    
    return text

######### CREATING DATA FRAME WITH ALL JOB OPPORTUNITIES URLS #####

In [258]:
######## functions ###########
def Salary(soup):
    try:
        salary = soup.find( attrs = {'class' : 'jobsearch-JobMetadataHeader-item'})
        return  salary.contents
    except:
        return 'Not Informed'
    
def Job_Type(soup):
    try:
        rawcode = requests.get(url)
        soup = BeautifulSoup(rawcode.text, "html.parser")
        job_type = soup.find( attrs = {'class' : 'jobsearch-JobMetadataHeader-item icl-u-xs-mt--xs'})
        return  job_type.contents
    except:
        return 'Not Informed'


In [ ]:
########################################
##### Loop for all the total pages #####
########################################

#num_pages_indeed = 1 
df_base = pd.DataFrame()

for i in range(1, num_pages_indeed+1):
    # generate the URL
    url = ''.join([url_indeed, '&start=', str(i*10)])
    #print(url)

    # get  LXML
    rawcode = requests.get(url)
    soup_lxml = BeautifulSoup(rawcode.text, "lxml")
    
    
    #Extracting the Salary


    # pick out all the "div" with "class="job-row"
    divs = soup_lxml.findAll("div")
    job_divs = [jp for jp in divs if not jp.get('class') is None
                    and 'row' in jp.get('class')]

    
    for job in job_divs:
        try:
            
            ix = job.index
            # job id
            id = job.get('data-jk', None)
            #print(id)
            
            # job link related to job id
            link = BASE_URL_indeed + '/viewjob'+ '?jk=' + id
            #link = 'https://ca.indeed.com/Data-Scientist-jobs-in-Canada?vjk=' + id
            #print(link)
            
            # job title
            title = job.find('a', attrs={'data-tn-element': 'jobTitle'}).attrs['title']
            #print(title)
            
            #sja = 'sja'+ str(count)
            #href = BASE_URL_indeed + job.find('a', attrs={'id': sja}).attrs['href']
            
            
            #print(href)
            #count = count + 1
            # job company
            company = job.find('span', {'class': 'company'}).text.strip()
            #print(company)
            
            # job location
            location = job.find('span', {'class': 'location'}).text.strip()
            #print(location)
            #city, province
            city,province = location.split(',')
            
            ### html.parser - Felipe
            r = requests.get(link)
            soup_html = BeautifulSoup(r.text, "html.parser")
            #print(soup_html)
            
            # salary
            salary = Salary(soup_html)
            #print('salary:' + salary)
            
            # job type
            jobtype = Job_Type(soup_html)

            
            #Extracting the Job Title
            position = soup_html.find(attrs = {'class' : 'icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title'})
            
            
           
            # job Requirements
            try:   
                #page = requests.get(link)
                #soup = BeautifulSoup(page.text, 'html.parser')
                text  = soup_html.get_text()
                jobrequirements = text
                #jobrequirements = text_cleaner(text)
                
            except:

                jobrequirements = 'Forbidden'

            
        except:
            continue

        df_base = df_base.append({'ix': ix,
                                  'Job_ID': id,
                                  'JobTitle': title,
                                  'JobType':jobtype,
                                  'CompanyName': company,
                                  'Salary' : salary ,
                                  'Position':position,
                                  'City': city,
                                  'Province':province,
                                  'JobRequirements':jobrequirements,
                                  'Date': now_str,
                                  'From':"Indeed",
                                  'JobLink':'link'},ignore_index=True)

print(df_base.head())


In [ ]:

df_base.to_csv('out.csv')

In [ ]:
############# READING DATAFRAME AND SCRAPING JOB DESCRIPTION ############

In [ ]:
#df_base.JobRequirements.values

In [ ]:
skill_source = pd.read_excel('ds.xlsx', sheet_name='skill')
kw = pd.read_excel('ds_kw.xlsx')


In [ ]:

# Functions
def find_skill(skill, jobReq ):
    if skill in jobReq:
        return True
    else:
        return False

In [ ]:
df_skill_aux = pd.DataFrame(columns=['Job_ID', 'KeyWord', 'Skill_ID'])


In [ ]:
df_skill_aux

In [ ]:
z = 0
for index, row in df_base.iterrows():
    k = kw.loc[kw.JobTitle == row["JobTitle"], ['KeyWord', 'Skill_ID']]
    jobReq = row["JobRequirements"]
    job_ID = row["Job_ID"]

    for idx, skills in k.iterrows():
        skill = ' ' + skills["KeyWord"]
        skill_ID = skills["Skill_ID"]
        func = find_skill(skill, jobReq)
        if func:
            # print(job_ID, skill, skill_ID)
            df_skill_aux.loc[z] = [job_ID, skill, skill_ID]
            z += 1




In [ ]:
df_skill_aux

In [ ]:
# Creating df_tmp
df_tmp = pd.DataFrame()


In [ ]:
#df_tmp = df_base.merge(df_skill_aux, how='left', left_on='Job_ID', right_on='ID')
df_tmp = df_base.merge(df_skill_aux, how='left', left_on='Job_ID', right_on='Job_ID')
df_tmp.head()

In [ ]:
############# Treating null values #################
values = {'Skill_ID': '-1', 'KeyWord': "Not found"}
df_tmp = df_tmp.fillna(value=values)

In [ ]:
df_tmp['Skill_ID'] = df_tmp['Skill_ID'].astype(int)

In [ ]:
df_tmp.head()

In [ ]:
skill_source

In [ ]:
####### Use this in case we want to filter jobs that does not match with keywords

#filters Skill_ID not identified
#df_filtered = df_tmp.query('Skill_ID>0')
#df_filtered.head()



In [241]:
#drop description colum to avoid a big data frame
df_tmp.drop(columns=['JobRequirements'])

,City,CompanyName,Date,From,JobLink,JobTitle,JobType,Job_ID,Position,Province,Salary,ix,KeyWord,Skill_ID
0,Toronto,Scotiabank,11/16/2018,Indeed,link,"Data Scientist Associate Director - Toronto, ON",Not Informed,2a924ec145aa9cac,"<h3 class=""icl-u-xs-mb--xs icl-u-xs-mt--none j...",ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Not found,-1
1,Toronto,Flipp,11/16/2018,Indeed,link,Data Scientist,Not Informed,86e0e9a1dab7acf3,"<h3 class=""icl-u-xs-mb--xs icl-u-xs-mt--none j...",ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Math,6
2,Toronto,Agility consulting Inc,11/16/2018,Indeed,link,Data Scientist,Not Informed,6e87ae66a10b3167,"<h3 class=""icl-u-xs-mb--xs icl-u-xs-mt--none j...",ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Math,6
3,St. Catharines,SomaDetect,11/16/2018,Indeed,link,Data Scientist (Deep Learning),Not Informed,17972833a76841f6,"<h3 class=""icl-u-xs-mb--xs icl-u-xs-mt--none j...",ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Not found,-1
4,Toronto,CIBC,11/16/2018,Indeed,link,Senior Data Scientist - Analytics Research,Not Informed,53e47810168e80a1,"<h3 class=""icl-u-xs-mb--xs icl-u-xs-mt--none j...",ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Not found,-1
5,Mississauga,Xylem,11/16/2018,Indeed,link,Data Scientist,Not Informed,3625131c23ab13f2,"<h3 class=""icl-u-xs-mb--xs icl-u-xs-mt--none j...",ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Math,6
6,Mississauga,Xylem,11/16/2018,Indeed,link,Data Scientist,Not Informed,3625131c23ab13f2,"<h3 class=""icl-u-xs-mb--xs icl-u-xs-mt--none j...",ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",big data,3
7,Toronto,Munich Re,11/16/2018,Indeed,link,Data Scientist / Actuary,Not Informed,96f5b2aafa9de20e,"<h3 class=""icl-u-xs-mb--xs icl-u-xs-mt--none j...",ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Not found,-1
8,Montréal,Stratuscent,11/16/2018,Indeed,link,Computational Chemistry Data Scientist,Not Informed,1510fd89bfd9e25b,"<h3 class=""icl-u-xs-mb--xs icl-u-xs-mt--none j...",QC,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Not found,-1


In [242]:

df_jobs = df_tmp.merge(skill_source, left_on='Skill_ID', right_on='Skill_ID', how='left')
#df_jobs = df_filtered.merge(skill_source, left_on='Skill_ID', right_on='Skill_ID', how='left')

In [243]:
df_jobs

,City,CompanyName,Date,From,JobLink,JobRequirements,JobTitle,JobType,Job_ID,Position,Province,Salary,ix,KeyWord,Skill_ID,Skill,Category
0,Toronto,Scotiabank,11/16/2018,Indeed,link,\n\nData Scientist Associate Director - Toront...,"Data Scientist Associate Director - Toronto, ON",Not Informed,2a924ec145aa9cac,"<h3 class=""icl-u-xs-mb--xs icl-u-xs-mt--none j...",ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Not found,-1,not identified,Not Identified
1,Toronto,Flipp,11/16/2018,Indeed,link,"\n\nData Scientist - Toronto, ON - Indeed.com\...",Data Scientist,Not Informed,86e0e9a1dab7acf3,"<h3 class=""icl-u-xs-mb--xs icl-u-xs-mt--none j...",ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Math,6,Mathematcs,Education
2,Toronto,Agility consulting Inc,11/16/2018,Indeed,link,"\n\nData Scientist - Toronto, ON - Indeed.com\...",Data Scientist,Not Informed,6e87ae66a10b3167,"<h3 class=""icl-u-xs-mb--xs icl-u-xs-mt--none j...",ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Math,6,Mathematcs,Education
3,St. Catharines,SomaDetect,11/16/2018,Indeed,link,\n\nData Scientist (Deep Learning) - St. Catha...,Data Scientist (Deep Learning),Not Informed,17972833a76841f6,"<h3 class=""icl-u-xs-mb--xs icl-u-xs-mt--none j...",ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Not found,-1,not identified,Not Identified
4,Toronto,CIBC,11/16/2018,Indeed,link,\n\nSenior Data Scientist - Analytics Research...,Senior Data Scientist - Analytics Research,Not Informed,53e47810168e80a1,"<h3 class=""icl-u-xs-mb--xs icl-u-xs-mt--none j...",ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Not found,-1,not identified,Not Identified
5,Mississauga,Xylem,11/16/2018,Indeed,link,"\n\nData Scientist - Mississauga, ON - Indeed....",Data Scientist,Not Informed,3625131c23ab13f2,"<h3 class=""icl-u-xs-mb--xs icl-u-xs-mt--none j...",ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Math,6,Mathematcs,Education
6,Mississauga,Xylem,11/16/2018,Indeed,link,"\n\nData Scientist - Mississauga, ON - Indeed....",Data Scientist,Not Informed,3625131c23ab13f2,"<h3 class=""icl-u-xs-mb--xs icl-u-xs-mt--none j...",ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",big data,3,BigData,Hard
7,Toronto,Munich Re,11/16/2018,Indeed,link,"\n\nData Scientist / Actuary - Toronto, ON - I...",Data Scientist / Actuary,Not Informed,96f5b2aafa9de20e,"<h3 class=""icl-u-xs-mb--xs icl-u-xs-mt--none j...",ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Not found,-1,not identified,Not Identified
8,Montréal,Stratuscent,11/16/2018,Indeed,link,\n\nComputational Chemistry Data Scientist - M...,Computational Chemistry Data Scientist,Not Informed,1510fd89bfd9e25b,"<h3 class=""icl-u-xs-mb--xs icl-u-xs-mt--none j...",QC,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Not found,-1,not identified,Not Identified


In [244]:
df_jobs = df_jobs.drop_duplicates(subset={'Job_ID','Skill_ID'}, keep='first', inplace=False)

In [245]:
df_jobs.head()

,City,CompanyName,Date,From,JobLink,JobRequirements,JobTitle,JobType,Job_ID,Position,Province,Salary,ix,KeyWord,Skill_ID,Skill,Category
0,Toronto,Scotiabank,11/16/2018,Indeed,link,\n\nData Scientist Associate Director - Toront...,"Data Scientist Associate Director - Toronto, ON",Not Informed,2a924ec145aa9cac,"<h3 class=""icl-u-xs-mb--xs icl-u-xs-mt--none j...",ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Not found,-1,not identified,Not Identified
1,Toronto,Flipp,11/16/2018,Indeed,link,"\n\nData Scientist - Toronto, ON - Indeed.com\...",Data Scientist,Not Informed,86e0e9a1dab7acf3,"<h3 class=""icl-u-xs-mb--xs icl-u-xs-mt--none j...",ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Math,6,Mathematcs,Education
2,Toronto,Agility consulting Inc,11/16/2018,Indeed,link,"\n\nData Scientist - Toronto, ON - Indeed.com\...",Data Scientist,Not Informed,6e87ae66a10b3167,"<h3 class=""icl-u-xs-mb--xs icl-u-xs-mt--none j...",ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Math,6,Mathematcs,Education
3,St. Catharines,SomaDetect,11/16/2018,Indeed,link,\n\nData Scientist (Deep Learning) - St. Catha...,Data Scientist (Deep Learning),Not Informed,17972833a76841f6,"<h3 class=""icl-u-xs-mb--xs icl-u-xs-mt--none j...",ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Not found,-1,not identified,Not Identified
4,Toronto,CIBC,11/16/2018,Indeed,link,\n\nSenior Data Scientist - Analytics Research...,Senior Data Scientist - Analytics Research,Not Informed,53e47810168e80a1,"<h3 class=""icl-u-xs-mb--xs icl-u-xs-mt--none j...",ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Not found,-1,not identified,Not Identified


In [246]:
df_final=df_jobs.to_csv('out.csv')